## Ant-AI Prototype Notebook

#### Library Imports, Notebook Setup

Includes Imports of source code packages. Namely the following packages are used in the prototype:
```
crewai
openai
yaml
pydantic
os
random
```

- CSV Logger Setup:
``logger = Logger.setup_logger(<Logging CSV File Location>)``
- CSV Logger Usage:
```logger.info('Logging CrewOutput Item', extra={'crew_output': <Crew Output Variable>})```

In [1]:
import os
import random
import asyncio

from pydantic import BaseModel

from ant_ai import Agents
from ant_ai import Logger

In [ ]:
# Reload ant_ai
import importlib
importlib.reload(Agents)

#### User Inputs

Edit Shown User Input Variables Here:

- ``LaymanPrompt``    : Initial High Level Task Definition.
- ``Persona``        : Persona To be followed for Task.
- ``Constraints``     : Task Hard Constraints to be followed through prompt optimization.

In [3]:
LaymanPrompt="""You are meant to re-write the text given to you in a simple, direct, and clear style.Follow these rules for communication:  
1. **Use Simple Language**: Write plainly with short, straightforward sentences.  
   - Example: "I need help with this issue."  

2. **Avoid AI-Giveaway Phrases**: Don't use clichés like "dive into" or "unleash your potential."  
   - Avoid: "Let's dive into this game-changing solution."  
   - Use instead: "Here's how it works."  

3. **Be Direct and Concise**: Remove unnecessary words and get to the point.  
   - Example: "We should meet tomorrow."  

4. **Maintain a Natural Tone**: Write as you would speak. It's fine to start sentences with "and" or "but."  
   - Example: "And that's why it matters."  

5. **Avoid Marketing Language**: Don't use hype or promotional terms.  
   - Avoid: "This revolutionary product will transform your life."  
   - Use instead: "This product can help you."  

6. **Keep It Honest**: Be real and avoid forced friendliness.  
   - Example: "I don't think that's the best idea."  

7. **Simplify Grammar**: Perfect grammar isn't necessary. Casual styles, like lowercase "i," are fine.  
   - Example: "i guess we can try that."  

8. **Avoid Fluff**: Leave out unnecessary adjectives and adverbs.  
   - Example: "We finished the task."  

9. **Focus on Clarity**: Make your writing easy to understand.  

Always ensure your communication is approachable, clear, and practical. 
"""
Persona="A LinkedIn Influencer"
Constraints="""
Ensure that the response includes reasoning pathways that are actionable and informative.
"""

#### Structured Output Base Models

This section includes the pydantic Base Model Defintions for Crew AI Structured Outputs.

In [4]:
from pydantic import BaseModel

#Dynamic Agents Creation

class Agent(BaseModel):
    agent_name: str
    agent_goal: str
    agent_backstory: str
    agent_task: str

class AgentCreation(BaseModel):
    agents: list[Agent]

#Critique Generation

class CritiqueReport(BaseModel):
    SpecializedGuidance : str
    SpecializedCritique : str

#Prompt Evaluation

class GeneralEvaluation(BaseModel):
    ScoreOutof90 : int

class ReasoningEvaluation(BaseModel):
    ScoreOutof70 : int

class CritiqueEvaluation(BaseModel):
    RefinedPromptCritique : str
    ActionableSteps : str

#Validy Check

class ValidityCheck(BaseModel):
    Validity : bool
    Reason : str

#### Crews Initialization

Crew Initialization is done with following details:
- Agent Defintions collected from ``ant-ai/defintions/AgentDef/``
- Task Defintions collected from ``ant-ai/defintions/TaskDef/``
- Logging Objects Saved to ``ant-ai/logs/``
- Agent LLM Temperatures Initialized With Each Crew Initialization.

In [ ]:
base_dir = os.path.dirname(os.path.abspath(""))
print(base_dir)

In [6]:
InitialPromptCrew = Agents.GetCrew(
    AgentYamlFile = os.path.join(base_dir, "definitions", "AgentDef", "MasterAgent.yaml"),
    TaskYamlFile = os.path.join(base_dir, "definitions", "TaskDef", "PromptGeneration.yaml"),
    OutputFile = os.path.join(base_dir, "logs", "InitialMasterLogs.txt"),
    LLM=Agents.LLM(
    model="gpt-4o-mini",api_key=os.getenv("OPENAI_API_KEY"),temperature=1
    ),
    verbose=True,
    allow_delegation=True,
)

PromptMutationCrew = Agents.GetCrew(
    AgentYamlFile = os.path.join(base_dir, "definitions", "AgentDef", "MutationAgent.yaml"),
    TaskYamlFile = os.path.join(base_dir, "definitions", "TaskDef", "MutationTask.yaml"),
    OutputFile = os.path.join(base_dir, "logs", "PromptMutationLogs.txt"),
    LLM=Agents.LLM(
    model="gpt-4o-mini",api_key=os.getenv("OPENAI_API_KEY"),temperature=0.8
    ),
    verbose=False,
    allow_delegation=True,
)

PromptCrossoverCrew = Agents.GetCrew(
    AgentYamlFile = os.path.join(base_dir, "definitions", "AgentDef", "CrossoverAgent.yaml"),
    TaskYamlFile = os.path.join(base_dir, "definitions", "TaskDef", "CrossoverTask.yaml"),
    OutputFile = os.path.join(base_dir, "logs", "CrossoverLogs.txt"),
    LLM=Agents.LLM(
    model="gpt-4o-mini",api_key=os.getenv("OPENAI_API_KEY"),temperature=0.7
    ),
    verbose=False,
    allow_delegation=True,
)

PromptGeneralEvaluationCrew = Agents.GetCrew(
    AgentYamlFile = os.path.join(base_dir, "definitions", "AgentDef", "EvaluationAgent.yaml"),
    TaskYamlFile = os.path.join(base_dir, "definitions", "TaskDef", "GeneralEvaluationTask.yaml"),
    OutputFile = os.path.join(base_dir, "logs", "EvaluationLogs.txt"),
    LLM=Agents.LLM(
    model="gpt-4o-mini",api_key=os.getenv("OPENAI_API_KEY"),temperature=0.2
    ),
    verbose=False,
    allow_delegation=True,
    Pydantic=GeneralEvaluation
)

PromptReasoningEvaluationCrew = Agents.GetCrew(
    AgentYamlFile = os.path.join(base_dir, "definitions", "AgentDef", "EvaluationAgent.yaml"),
    TaskYamlFile = os.path.join(base_dir, "definitions", "TaskDef", "ReasoningEvaluationTask.yaml"),
    OutputFile = os.path.join(base_dir, "logs", "EvaluationLogs.txt"),
    LLM=Agents.LLM(
    model="gpt-4o-mini",api_key=os.getenv("OPENAI_API_KEY"),temperature=0.2
    ),
    verbose=False,
    allow_delegation=True,
    Pydantic=ReasoningEvaluation
)

PromptCritiqueEvaluationCrew = Agents.GetCrew(
    AgentYamlFile = os.path.join(base_dir, "definitions", "AgentDef", "EvaluationAgent.yaml"),
    TaskYamlFile = os.path.join(base_dir, "definitions", "TaskDef", "EvaluationCritiqueTask.yaml"),
    OutputFile = os.path.join(base_dir, "logs", "EvaluationLogs.txt"),
    LLM=Agents.LLM(
    model="gpt-4o-mini",api_key=os.getenv("OPENAI_API_KEY"),temperature=0.6
    ),
    verbose=False,
    allow_delegation=True,
    Pydantic=CritiqueEvaluation
)

DynamicAgentCreationCrew = Agents.GetCrew(
    AgentYamlFile = os.path.join(base_dir, "definitions", "AgentDef", "DynamicAgentManager.yaml"),
    TaskYamlFile = os.path.join(base_dir, "definitions", "TaskDef", "AgentCreation.yaml"),
    OutputFile = os.path.join(base_dir, "logs", "DynamicAgentCreationLogs.txt"),
    LLM=Agents.LLM(
    model="gpt-4o-mini",api_key=os.getenv("OPENAI_API_KEY"),temperature=0.3
    ),
    verbose=True,
    allow_delegation=True,
    Pydantic=AgentCreation
)

InitialCritiqueCrew = Agents.GetCrew(
    AgentYamlFile = os.path.join(base_dir, "definitions", "AgentDef", "CritiquePromptAgent.yaml"),
    TaskYamlFile = os.path.join(base_dir, "definitions", "TaskDef", "CritiquePromptGeneration.yaml"),
    OutputFile = os.path.join(base_dir, "logs", "InitialMetaMasterLogs.txt"),
    LLM=Agents.LLM(
    model="gpt-4o-mini",api_key=os.getenv("OPENAI_API_KEY"),temperature=0.6
    ),
    verbose=False,
    allow_delegation=True,
    Pydantic=CritiqueReport
)

FinalUnificationCrew = Agents.GetCrew(
    AgentYamlFile = os.path.join(base_dir, "definitions", "AgentDef", "CombinationAgent.yaml"),
    TaskYamlFile = os.path.join(base_dir, "definitions", "TaskDef", "CombinationTask.yaml"),
    OutputFile = os.path.join(base_dir, "logs", "CombinationLogs.txt"),
    LLM=Agents.LLM(
    model="gpt-4o-mini",api_key=os.getenv("OPENAI_API_KEY"),temperature=0.9
    ),
    verbose=True,
    allow_delegation=True,
)

Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed


#### Initial Combined Prompt Generation

Generation of ``Initial Prompt`` to be passed for ``Dynamic Agents Generation`` and for the ``Initial Population Generation``.

In [7]:
Validity,Reason=Agents.ValidityCheck(
    LaymanPrompt,
    Persona,
    Constraints,
    os.path.join(base_dir, "definitions", "TaskDef", "SystemGuardrail.yaml"),
    ValidityCheck)

True
The provided prompt requests the superintelligent AI to rewrite text in a simple and clear style while adhering to specific guidelines intended to improve clarity and engagement. It doesn't contain harmful or malicious content, nor does it aim to elicit inappropriate outputs. The input also includes a defined persona and constraints, making it a valid request for optimization that could enhance user communication without risking security or wasting computational resources.


In [9]:
PromptCrew=InitialPromptCrew.kickoff(inputs={"LaymanPrompt":LaymanPrompt,"Persona":Persona,"Constraints":Constraints})
InitialPrompt=PromptCrew.raw

# Agent: Head of Prompt Engineering
## Task: <ant-task>
You are meant to re-write the text given to you in a simple, direct, and clear style.Follow these rules for communication:  
1. **Use Simple Language**: Write plainly with short, straightforward sentences.  
   - Example: "I need help with this issue."  

2. **Avoid AI-Giveaway Phrases**: Don't use clichés like "dive into" or "unleash your potential."  
   - Avoid: "Let's dive into this game-changing solution."  
   - Use instead: "Here's how it works."  

3. **Be Direct and Concise**: Remove unnecessary words and get to the point.  
   - Example: "We should meet tomorrow."  

4. **Maintain a Natural Tone**: Write as you would speak. It's fine to start sentences with "and" or "but."  
   - Example: "And that's why it matters."  

5. **Avoid Marketing Language**: Don't use hype or promotional terms.  
   - Avoid: "This revolutionary product will transform your life."  
   - Use instead: "This product can help you."  

6. **Keep It 

#### Dynamic Agents Generation

Generation and formatting of Dynamic Agents on the Basis of Pydantic Base Models.
- ``AgentCount`` Defines the list of Dynamic Agent Objects to be created by crew.

In [10]:
AgentCount=3
AgentsListRaw=DynamicAgentCreationCrew.kickoff(inputs={"InitialTask":InitialPrompt,"AgentCount":str(AgentCount)})

# Agent: Dynamic Agent Manager
## Task: <ant-task>
<assisstant-context>  
You need to rewrite a text clearly and simply while adopting the tone of a LinkedIn influencer, maintaining directness and approachability.  
</assisstant-context>  

<assisstant-task>  
Rewrite the provided text in a simple, direct, and clear style, ensuring that it communicates effectively without complex language or marketing hype.  
</assisstant-task>  

<assisstant-constraints>  
- Use simple language with short sentences.  
- Avoid clichés and promotional phrases.  
- Be direct and concise, cutting out unnecessary words.  
- Maintain a natural tone, allowing informalities.  
- Write honestly without forced friendliness.  
- Simplify grammar usage.  
- Exclude unnecessary fluff or adjectives.  
- Focus on clarity and ease of understanding.  
</assisstant-constraints>  

<assisstant-reasoning>  
Utilize clear and structured processes for rewriting that will ensure adherence to the constraints and persona's vo

In [13]:
AgentInputs = []
for agent in AgentsListRaw.pydantic.agents:
    AgentInputs.append(
        {
            "AgentName":agent.agent_name,
            'AgentBackstory':agent.agent_backstory,
            'AgentGoal':agent.agent_goal,
            'AgentTask':agent.agent_task,
            "InitialTask":InitialPrompt
        }
    )

In [14]:
def get_agent(agent_inputs):
    agent = Agents.Agent(
        role = agent_inputs['AgentName'],
        goal = agent_inputs['AgentGoal'],
        backstory = agent_inputs['AgentBackstory'],
        verbose = False,
        llm = Agents.LLM(
        model="gpt-4o-mini",api_key=os.getenv("OPENAI_API_KEY")
        ),
        temperature = 0.4
    )
    return agent

In [15]:
AgentInputs

[{'AgentName': 'Clarity Specialist',
  'AgentBackstory': 'You are a communication expert with a background in linguistics and experience in editing technical documents. Your passion lies in making complicated ideas accessible to everyone, and you have worked with various organizations to refine their messaging for broader audiences.',
  'AgentGoal': 'As a Clarity Specialist, your goal is to enhance the clarity of the task definition in `<ant-task>` by simplifying complex phrases and ensuring the message is straightforward and easy to understand.',
  'AgentTask': 'Review the task definition in `<ant-task>` methodically, identifying areas where language can be simplified. Focus on removing jargon and complex phrases, ensuring that the revised task is direct and clear. Provide a revised version that maintains the original intent while enhancing readability.',
  'InitialTask': '<assisstant-context>\nAs a LinkedIn Influencer, participate in refining the communication style of a given text t

### Genetic Algorithm

#### GA Setup

Includes the Primary GA Setup Including the following:
- ``PromptGene`` Class Definition.
- ``Gene Generation`` and ``Gene Management`` Functions.
- ``Population Generation`` for GA.

In [16]:
class PromptGene:
    
    def __init__(self,initial_prompt="", agent_guidance="", agent_critique="", refined_prompt="", result="", action_steps="", general_score=0, reasoning_score=0, refined_prompt_critique=""):
        self.InitialPrompt = initial_prompt
        self.AgentGuidance = agent_guidance
        self.AgentCritique = agent_critique
        self.RefinedPrompt = refined_prompt
        self.Result = result
        self.ActionSteps = action_steps
        self.GeneralScore = general_score
        self.ReasoningScore = reasoning_score
        self.RefinedPromptCritique = refined_prompt_critique
    
    

    def display(self):
        print(f"\n\n\n\n\nInitial Prompt: {self.InitialPrompt}")
        print(f"\n\n\n\n\nAgent Guidance: {self.AgentGuidance}")
        print(f"\n\n\n\n\nAgent Critique: {self.AgentCritique}")
        print(f"\n\n\n\n\nRefined Prompt: {self.RefinedPrompt}")
        print(f"\n\n\n\n\nResult: {self.Result}")
        print(f"\n\n\n\n\nAction Steps: {self.ActionSteps}")
        print(f"\n\n\n\n\nGeneral Score: {self.GeneralScore}")
        print(f"\nReasoning Score: {self.ReasoningScore}")
        print(f"\nTotal Score: {self.GeneralScore+self.ReasoningScore}")    
        print(f"\n\n\n\n\nRefined Prompt Critique: {self.RefinedPromptCritique}")


    def get(self):
        inputArr={
            'InitialTask': self.InitialPrompt, 
            'SpecializedGuidance': self.AgentGuidance,
            'RefinedTask': self.RefinedPrompt, 
            'RefinedTaskOutput': self.Result
        }
        return inputArr

    def generateEval(self):
        GeneralScore=PromptGeneralEvaluationCrew.kickoff(self.get())
        ReasoningScore=PromptReasoningEvaluationCrew.kickoff(self.get())
        inputs=self.get()
        CritiqueEval=PromptCritiqueEvaluationCrew.kickoff(inputs={
            'InitialTask': inputs['InitialTask'],
            'SpecializedGuidance': inputs['SpecializedGuidance'],
            'RefinedTask': inputs['RefinedTask'],
            'RefinedTaskOutput': inputs['RefinedTaskOutput'],
            'GeneralEvaluation': GeneralScore.raw,
            'ReasoningEvaluation': ReasoningScore.raw
        })
        self.GeneralScore=GeneralScore.pydantic.ScoreOutof90
        self.ReasoningScore=ReasoningScore.pydantic.ScoreOutof70
        self.ActionSteps=CritiqueEval.pydantic.ActionableSteps
        self.RefinedPromptCritique=CritiqueEval.pydantic.RefinedPromptCritique
        return self.GeneralScore+self.ReasoningScore


In [17]:
def GenerateGene(InitialPrompt, CrtiqueReport, LayerCrew):
    GenePromptResult=LayerCrew.kickoff(inputs={"InitialTask":InitialPrompt,"SpecializedGuidance":CrtiqueReport.SpecializedGuidance,"SpecializedCritique":CrtiqueReport.SpecializedCritique})
    GenePrompt=GenePromptResult.raw
    GeneOutput=Agents.InvokeGpt4oMini(prompt=GenePrompt)
    Gene=PromptGene(InitialPrompt,CrtiqueReport.SpecializedGuidance,CrtiqueReport.SpecializedCritique,GenePrompt,GeneOutput)
    Gene.generateEval()
    return Gene

In [18]:
def GenerateGeneFromPrompt(PrimaryPrompt,AgentInputs):
    UpdatedCritique=InitialCritiqueCrew.kickoff(inputs={
        'InitialTask':PrimaryPrompt,
        'AgentName':AgentInputs['AgentName'],
        'AgentBackstory':AgentInputs['AgentBackstory'],
        'AgentGoal':AgentInputs['AgentGoal'],
        'AgentTask':AgentInputs['AgentTask']
    })
    PromptGenerator=Agents.GetCrewWithAgent(
        Agent=get_agent(AgentInputs),
        TaskYamlFile= os.path.join(base_dir, "definitions", "TaskDef", "SpecializedPromptGeneration.yaml"),
        OutputFile= os.path.join(base_dir, "logs", "SpecializedPromptLogs.txt"),
    )
    NewGene=GenerateGene(PrimaryPrompt,UpdatedCritique.pydantic,PromptGenerator)
    return NewGene

In [19]:
async def run_tasks_async_custom(Agents, InputPerAgent, InputData):
    tasks = []
    num_agents = len(Agents)

    for i in range(len(InputData)):
        agent_index = (i // InputPerAgent) % num_agents
        tasks.append(Agents[agent_index].kickoff_async(inputs=InputData[i]))

    results = await asyncio.gather(*tasks)
    return results

In [20]:

async def generate_population_async(initial_prompt, agent_inputs , population_size):    

    InputData=[]
    PromptGenerators=[]
    for agent_input in agent_inputs:
        for i in range(population_size):
            InputData.append({
                'InitialTask':initial_prompt,
                'AgentName':agent_input['AgentName'],
                'AgentBackstory':agent_input['AgentBackstory'],
                'AgentGoal':agent_input['AgentGoal'],
                'AgentTask':agent_input['AgentTask']
            })
        PromptGenerator=Agents.GetCrewWithAgent(
            Agent=get_agent(agent_input),
            TaskYamlFile= os.path.join(base_dir, "definitions", "TaskDef", "SpecializedPromptGeneration.yaml"),
            OutputFile= os.path.join(base_dir, "logs", "SpecializedPromptLogs.txt"),
            Async=True
        )
        PromptGenerators.append(PromptGenerator)

    UpdatedCritiques = await InitialCritiqueCrew.kickoff_for_each_async(inputs=InputData)


    InputData=[]
    for i in range(len(UpdatedCritiques)):
        InputData.append({
            'InitialTask':initial_prompt,
            'SpecializedGuidance':UpdatedCritiques[i].pydantic.SpecializedGuidance,
            'SpecializedCritique':UpdatedCritiques[i].pydantic.SpecializedCritique
        })
    UpdatedPrompts=await run_tasks_async_custom(PromptGenerators,population_size,InputData)

    UpdatedRawPrompts=[UpdatedPrompts[i].raw for i in range(len(UpdatedCritiques))]
    GeneOutputs=await Agents.gather_responses(UpdatedRawPrompts)

    NewGenes=[]
    for i in range(len(UpdatedCritiques)):
        NewGene=PromptGene(initial_prompt,UpdatedCritiques[i].pydantic.SpecializedGuidance,UpdatedCritiques[i].pydantic.SpecializedCritique,UpdatedPrompts[i].raw,GeneOutputs[i])
        NewGenes.append(NewGene)
    
    InputData=[]
    for i in range(len(NewGenes)):
        InputData.append(NewGenes[i].get())

    GeneralScores= await PromptGeneralEvaluationCrew.kickoff_for_each_async(inputs=InputData)
    ReasoningScores= await  PromptReasoningEvaluationCrew.kickoff_for_each_async(inputs=InputData)

    InputData=[]
    for i in range(len(NewGenes)):
        InputData.append({
            'InitialTask': NewGenes[i].InitialPrompt,
            'SpecializedGuidance': NewGenes[i].AgentGuidance,
            'RefinedTask': NewGenes[i].RefinedPrompt,
            'RefinedTaskOutput': NewGenes[i].Result,
            'GeneralEvaluation': GeneralScores[i].raw,
            'ReasoningEvaluation': ReasoningScores[i].raw
        })

    
    CritiqueEvals=  await PromptCritiqueEvaluationCrew.kickoff_for_each_async(inputs=InputData)

    for i in range(population_size):
        NewGenes[i].GeneralScore=GeneralScores[i].pydantic.ScoreOutof90
        NewGenes[i].ReasoningScore=ReasoningScores[i].pydantic.ScoreOutof70
        NewGenes[i].RefinedPromptCritique=CritiqueEvals[i].pydantic.RefinedPromptCritique
        NewGenes[i].ActionSteps=CritiqueEvals[i].pydantic.ActionableSteps

    Genes=[]
    for i in range(len(agent_inputs)):
        AgentGenes=[]
        for y in range(population_size):
            AgentGenes.append(NewGenes[i*population_size+y])
        Genes.append(AgentGenes)

    
    return Genes

In [21]:
PopulationList=await generate_population_async(InitialPrompt, AgentInputs, 3)

Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed


In [38]:
PopulationList

[[<__main__.PromptGene at 0x230d007cf90>,

#### GA Functions

General GA Function Defintions are present including helper functions for ``Crossover``, ``Mutation`` & ``Termination``.

In [22]:
import streamlit as st

In [33]:
async def CrossoverMutatePrompts(PrimaryPopulation,SecondaryPopulation,MutationPopulation,AgentInputs):
    if len(PrimaryPopulation)!=len(SecondaryPopulation) or len(PrimaryPopulation)!=len(MutationPopulation):
        return None
    for i in range(len(AgentInputs)):
        if PrimaryPopulation[i].GeneralScore==0 or PrimaryPopulation[i].ReasoningScore==0:
            PrimaryPopulation[i].generateEval()
        if SecondaryPopulation[i].GeneralScore==0 or SecondaryPopulation[i].ReasoningScore==0:
            SecondaryPopulation[i].generateEval()
        if MutationPopulation[i].GeneralScore==0 or MutationPopulation[i].ReasoningScore==0:
            MutationPopulation[i].generateEval()

    tasks=[]
    PromptCrossoverCrewCopies=[PromptCrossoverCrew.copy() for _ in range(len(AgentInputs))]
    PromptMutationCrewCopies=[PromptMutationCrew.copy() for _ in range(len(AgentInputs))]
    for i in range(len(AgentInputs)):
        task=PromptCrossoverCrewCopies[i].kickoff_async(inputs={
            'InitialTask1':PrimaryPopulation[i].RefinedPrompt,
            'PromptCritique1':PrimaryPopulation[i].RefinedPromptCritique,
            'InitialTask2':SecondaryPopulation[i].RefinedPrompt,
            'PromptCritique2':SecondaryPopulation[i].RefinedPromptCritique,
        })
        tasks.append(task)
    for i in range(len(AgentInputs)):
        task=PromptMutationCrewCopies[i].kickoff_async(inputs={
            'InitialTask':MutationPopulation[i].RefinedPrompt,
            'PromptCritique':MutationPopulation[i].RefinedPromptCritique,
            'ActionSteps':MutationPopulation[i].ActionSteps
        })
        tasks.append(task)
    NewPrompts=await asyncio.gather(*tasks)
    CrossoverPrompts=NewPrompts[:len(AgentInputs)] 
    MutationPrompts=NewPrompts[len(AgentInputs):]    
    st.write("New Prompts Generated")
    
    tasks=[]
    InitialCritiqueCrewCopies1=[InitialCritiqueCrew.copy() for _ in range(len(AgentInputs))]
    InitialCritiqueCrewCopies2=[InitialCritiqueCrew.copy() for _ in range(len(AgentInputs))]
    for i in range(len(AgentInputs)):
        task=InitialCritiqueCrewCopies1[i].kickoff_async(inputs={
            'InitialTask':CrossoverPrompts[i].raw,
            'AgentName':AgentInputs[i]['AgentName'],
            'AgentBackstory':AgentInputs[i]['AgentBackstory'],
            'AgentGoal':AgentInputs[i]['AgentGoal'],
            'AgentTask':AgentInputs[i]['AgentTask']
        })
        tasks.append(task)
    for i in range(len(AgentInputs)):
        task=InitialCritiqueCrewCopies2[i].kickoff_async(inputs={
            'InitialTask':MutationPrompts[i].raw,
            'AgentName':AgentInputs[i]['AgentName'],
            'AgentBackstory':AgentInputs[i]['AgentBackstory'],
            'AgentGoal':AgentInputs[i]['AgentGoal'],
            'AgentTask':AgentInputs[i]['AgentTask']
        })
        tasks.append(task)
    UpdatedCritiques = await asyncio.gather(*tasks)
    CrossoverCritiques=UpdatedCritiques[:len(AgentInputs)]
    MutationCritiques=UpdatedCritiques[len(AgentInputs):]
    st.write("New Critiques Generated")
    
    PromptGenerator=[]
    for i in range(len(AgentInputs)):
        PromptGenerator.append(
            Agents.GetCrewWithAgent(
                Agent=get_agent(AgentInputs[i]),
                TaskYamlFile= os.path.join(base_dir, "definitions", "TaskDef", "SpecializedPromptGeneration.yaml"),
                OutputFile= os.path.join(base_dir, "logs", "SpecializedPromptLogs.txt"),
            )
        )
    
    PromptGeneratorCopies=[PromptGenerator[i].copy() for i in range(len(AgentInputs))]

    tasks=[]
    for i in range(len(AgentInputs)):
        task=PromptGenerator[i].kickoff_async(inputs={
            'InitialTask':CrossoverPrompts[i].raw,
            'SpecializedGuidance':CrossoverCritiques[i].pydantic.SpecializedGuidance,
            'SpecializedCritique':CrossoverCritiques[i].pydantic.SpecializedCritique
        })
        tasks.append(task)
    for i in range(len(AgentInputs)):
        task=PromptGeneratorCopies[i].kickoff_async(inputs={
            'InitialTask':MutationPrompts[i].raw,
            'SpecializedGuidance':MutationCritiques[i].pydantic.SpecializedGuidance,
            'SpecializedCritique':MutationCritiques[i].pydantic.SpecializedCritique
        })
        tasks.append(task)
    UpdatedPrompts=await asyncio.gather(*tasks)
    CrossoverRefinedPrompts=UpdatedPrompts[:len(AgentInputs)]
    MutationRefinedPrompts=UpdatedPrompts[len(AgentInputs):]
    st.write("New Gene Prompts Generated")
    UpdatedPromptsRaw=[UpdatedPrompts[i].raw for i in range(len(UpdatedPrompts))]

    GeneOutputs=await Agents.gather_responses(UpdatedPromptsRaw)
    CrossoverGeneOutputs=GeneOutputs[:len(AgentInputs)]
    MutationGeneOutputs=GeneOutputs[len(AgentInputs):]

    CrossoverGenes=[]
    MutationGenes=[]
    for i in range(len(AgentInputs)):
        NewGene=PromptGene(CrossoverPrompts[i].raw,CrossoverCritiques[i].pydantic.SpecializedGuidance,CrossoverCritiques[i].pydantic.SpecializedCritique,CrossoverRefinedPrompts[i].raw,CrossoverGeneOutputs[i])
        CrossoverGenes.append(NewGene)
    for i in range(len(AgentInputs)):
        NewGene=PromptGene(MutationPrompts[i].raw,MutationCritiques[i].pydantic.SpecializedGuidance,MutationCritiques[i].pydantic.SpecializedCritique,MutationRefinedPrompts[i].raw,MutationGeneOutputs[i])
        MutationGenes.append(NewGene)

    PromptGeneralEvaluationCrewCopies1=[PromptGeneralEvaluationCrew.copy() for _ in range(len(AgentInputs))]
    PromptGeneralEvaluationCrewCopies2=[PromptGeneralEvaluationCrew.copy() for _ in range(len(AgentInputs))]
    PromptReasoningEvaluationCrewCopies1=[PromptReasoningEvaluationCrew.copy() for _ in range(len(AgentInputs))]
    PromptReasoningEvaluationCrewCopies2=[PromptReasoningEvaluationCrew.copy() for _ in range(len(AgentInputs))]

    gen_eval_tasks=[]
    res_eval_tasks=[]

    for i in range(len(AgentInputs)):
        task=PromptGeneralEvaluationCrewCopies1[i].kickoff_async(inputs=CrossoverGenes[i].get())
        gen_eval_tasks.append(task)
        task=PromptReasoningEvaluationCrewCopies1[i].kickoff_async(inputs=CrossoverGenes[i].get())
        res_eval_tasks.append(task)
        
    AllTasks=gen_eval_tasks+res_eval_tasks
    Results=await asyncio.gather(*AllTasks)
    CrossoverGeneralScores=Results[:len(AgentInputs)]
    CrossoverReasoningScores=Results[len(AgentInputs):]

    gen_eval_tasks=[]
    res_eval_tasks=[]

    for i in range(len(AgentInputs)):
        task=PromptGeneralEvaluationCrewCopies2[i].kickoff_async(inputs=MutationGenes[i].get())
        gen_eval_tasks.append(task)
        task=PromptReasoningEvaluationCrewCopies2[i].kickoff_async(inputs=MutationGenes[i].get())
        res_eval_tasks.append(task)

    AllTasks=gen_eval_tasks+res_eval_tasks
    Results=await asyncio.gather(*AllTasks)
    MutationGeneralScores=Results[:len(AgentInputs)]
    MutationReasoningScores=Results[len(AgentInputs):]

    PromptCritiqueEvaluationCrewCopies1=[PromptCritiqueEvaluationCrew.copy() for _ in range(len(AgentInputs))]
    PromptCritiqueEvaluationCrewCopies2=[PromptCritiqueEvaluationCrew.copy() for _ in range(len(AgentInputs))]

    CritiqueEvalTasks=[]
    for i in range(len(AgentInputs)):
        inputs=CrossoverGenes[i].get()
        task=PromptCritiqueEvaluationCrewCopies1[i].kickoff_async(inputs={
            'InitialTask': inputs['InitialTask'],
            'SpecializedGuidance': inputs['SpecializedGuidance'],
            'RefinedTask': inputs['RefinedTask'],
            'RefinedTaskOutput': inputs['RefinedTaskOutput'],
            'GeneralEvaluation': CrossoverGeneralScores[i].raw,
            'ReasoningEvaluation': CrossoverReasoningScores[i].raw
        })
        CritiqueEvalTasks.append(task)
    for i in range(len(AgentInputs)):
        inputs=MutationGenes[i].get()
        task=PromptCritiqueEvaluationCrewCopies2[i].kickoff_async(inputs={
            'InitialTask': inputs['InitialTask'],
            'SpecializedGuidance': inputs['SpecializedGuidance'],
            'RefinedTask': inputs['RefinedTask'],
            'RefinedTaskOutput': inputs['RefinedTaskOutput'],
            'GeneralEvaluation': MutationGeneralScores[i].raw,
            'ReasoningEvaluation': MutationReasoningScores[i].raw
        })
        CritiqueEvalTasks.append(task)

    CritiqueEvals=await asyncio.gather(*CritiqueEvalTasks)

    CrossoverCritiqueEvals=CritiqueEvals[:len(AgentInputs)]
    MutationCritiqueEvals=CritiqueEvals[len(AgentInputs):]

    for i in range(len(AgentInputs)):
        CrossoverGenes[i].GeneralScore=CrossoverGeneralScores[i].pydantic.ScoreOutof90
        CrossoverGenes[i].ReasoningScore=CrossoverReasoningScores[i].pydantic.ScoreOutof70
        CrossoverGenes[i].RefinedPromptCritique=CrossoverCritiqueEvals[i].pydantic.RefinedPromptCritique
        CrossoverGenes[i].ActionSteps=CrossoverCritiqueEvals[i].pydantic.ActionableSteps
        MutationGenes[i].GeneralScore=MutationGeneralScores[i].pydantic.ScoreOutof90
        MutationGenes[i].ReasoningScore=MutationReasoningScores[i].pydantic.ScoreOutof70
        MutationGenes[i].RefinedPromptCritique=MutationCritiqueEvals[i].pydantic.RefinedPromptCritique
        MutationGenes[i].ActionSteps=MutationCritiqueEvals[i].pydantic.ActionableSteps
        
    return CrossoverGenes,MutationGenes

In [28]:
def CrossoverPrompt(PrimaryGene, SecondaryGene):
    if (PrimaryGene.GeneralScore==0 or PrimaryGene.ReasoningScore==0):
        PrimaryGene.generateEval()
    if (SecondaryGene.GeneralScore==0 or SecondaryGene.ReasoningScore==0):
        SecondaryGene.generateEval()
    NewPrompt=PromptCrossoverCrew.kickoff(inputs={
        'InitialTask1':PrimaryGene.RefinedPrompt,
        'PromptCritique1':PrimaryGene.RefinedPromptCritique,
        'InitialTask2':SecondaryGene.RefinedPrompt,
        'PromptCritique2':SecondaryGene.RefinedPromptCritique,
    })
    return NewPrompt.raw

def CrossoverGene(PrimaryGene,SecondaryGene,AgentInputs):
    NewPrompt=CrossoverPrompt(PrimaryGene,SecondaryGene)
    print("New Initial Prompt Generated")
    print("Generating Crossover Gene")
    NewGene=GenerateGeneFromPrompt(NewPrompt,AgentInputs)
    return NewGene

In [29]:
def MutatePrompt(PrimaryGene):
    if (PrimaryGene.GeneralScore==0 or PrimaryGene.ReasoningScore==0):
        PrimaryGene.generateEval()
    NewPrompt=PromptMutationCrew.kickoff(inputs={
        'InitialTask':PrimaryGene.RefinedPrompt,
        'PromptCritique':PrimaryGene.RefinedPromptCritique,
        'ActionSteps':PrimaryGene.ActionSteps
    })
    return NewPrompt.raw

def MutateGene(PrimaryGene,AgentInputs):
    NewPrompt=MutatePrompt(PrimaryGene)
    print("New Initial Prompt Generated")
    print("Generating Mutated Gene")
    MutatedGene=GenerateGeneFromPrompt(NewPrompt,AgentInputs)
    return MutatedGene
    

In [31]:
def ArrangePopulation(Population):
    # Sort the population by the 'score' attribute in descending order
    Population.sort(key=lambda obj: obj.GeneralScore+obj.ReasoningScore, reverse=True)
    return Population   

#### Main GA Definition

This Part Covers the Main GA Definition for the entire workflow. Taking the following Inputs:
- ``InitialPopulation``     : The Primary Population to be passed to GA for Optimization.
- ``MaxGenerations``        : Defintion of the number of generations for GA to run for optimization.
- ``MaxSelectionSize``      : Defines the Number of Genes to be present in a generation at a time, defining the termination criteria.
- ``CrossoverRate``         : Defines the Rate of Crossover in each Generation (0-1).
- ``MutationRate``          : Defines the Rate of Mutation in Each Generation (0-1).
- ``LayerAgentInputs``      : Structured Input object of Dynamic Agent Selected for GA Optimization. (Corresponding the Initial Population).

In [75]:
import random

def run_genetic_algorithm(InitialPopulation, MaxGenerations, SelectionSize, CrossoverRate,TopCarry,TopCarryRate, MutationRate, LayerAgentInputs):
    # Initialize the population with the initial gene and a mutated version of the gene
    population = InitialPopulation
    
    # Print the initial population
    print(f"--- Initial Population ---")
    print(f"Initial Population: {population}")
    print(f"Population Size: {len(population)}\n")
    print(f"Population before sorting: {population}")
    print(f"Population scores before sorting: {[gene.GeneralScore+gene.ReasoningScore for gene in population]}")
    population = ArrangePopulation(population)
    print(f"Population after sorting: {population}")
    print(f"Population scores after sorting: {[gene.GeneralScore+gene.ReasoningScore for gene in population]}")
    
    # Begin the evolutionary loop for MaxGenerations generations
    for generation in range(1, MaxGenerations + 1):
        print(f"\n############################")
        print(f"# Generation {generation}")
        print(f"############################")
        new_population = []
        # Step 1: Carry forward the entire population from the previous generation (Elitism)
        print(f"\nStep 1: Carry forward the entire last generation's population.")
        print(f"Population at start of generation {generation}: {population}")
        
        # Step 2: Generate new individuals by crossover and mutation until we meet or exceed the max population size
        print("\nStep 2: Performing crossover and mutation to introduce new offspring.")
                
        while True:
            # Select two different parents for crossover
            parent_1, parent_2 = random.sample(population, 2)  # Guarantees parent_1 != parent_2
            print(f"\nSelected Parents for Crossover: Parent_1 = {parent_1}, Parent_2 = {parent_2}")
            
            # Perform crossover with probability CrossoverRate
            if random.random() < CrossoverRate:
                print(f"Performing Crossover (Crossover Rate: {CrossoverRate})")
                crossed_gene = CrossoverGene(parent_1, parent_2, LayerAgentInputs)
                print(f"New Crossover Gene: {crossed_gene}")
                new_population.append(crossed_gene)
            else:
                print(f"Skipping Crossover (Crossover Rate: {CrossoverRate})")

            print(f"\nSelected Parents for Mutation: Parent = {population[0]}")
            # Step 3: Apply mutation with MutationRate probability
            if random.random() < MutationRate:
                print(f"\nMutating the Best Performing Gene (Mutation Rate: {MutationRate})")
                best_gene = population[0]
                mutated_gene = MutateGene(best_gene, LayerAgentInputs)
                new_population.append(mutated_gene)
                print(f"Mutated Gene: {mutated_gene}")
            else:
                print(f"Skipping Mutation (Mutation Rate: {MutationRate})")

            # Break the loop if the population has met or exceeded the MaxSelectionSize
            if len(new_population) >= SelectionSize:
                break  

        # Step 4: Trim TopCarry individuals from the population and add them to the new population
        print(f"\nStep 4: Trimming the Top {TopCarry} individuals from the population and adding them to the new population.")
        top_carry = population[:TopCarry]
        print(f"Top {TopCarry} Carry Individuals: {top_carry}")
        for gene in top_carry:
            if random.random() < TopCarryRate:
                new_population.append(gene)
                print (f"Adding Top Carry Individual: {gene}")
            else:
                print(f"Skipping Top Carry Individual: {gene}")
        print(f"New Population after adding Top Carry Individuals: {new_population}")
        population = new_population
        print(f"New Population : {population}")
        # Step 5: Sort the population by fitness (best to worst) and trim to MaxSelectionSize (elitism)
        print(f"\nStep 5: Sorting and trimming the population size to Top {SelectionSize}")
        print(f"Population before sorting: {population}")
        print(f"Population scores before sorting: {[gene.GeneralScore+gene.ReasoningScore for gene in population]}")
        population = ArrangePopulation(population)
        print(f"Population after sorting: {population}")
        print(f"Population scores after sorting: {[gene.GeneralScore+gene.ReasoningScore for gene in population]}")
        
        # If the population exceeds MaxSelectionSize, trim it down to the best MaxSelectionSize individuals
        if len(population) > SelectionSize:
            print(f"Trimming population from {len(population)} to {SelectionSize}")
            population = population[:SelectionSize]    
        print(f"Population at the end of generation {generation}: {population}")
        print(f"Population Size at the end of generation {generation}: {len(population)}\n")
    
    print(f"\n############################")
    print(f"# Final Generation Complete")
    print(f"############################")
    print(f"Final population: {population}")
    print(f"Final population scores: {[gene.GeneralScore+gene.ReasoningScore for gene in population]}")
    print(f"Final Population Size: {len(population)}")
    population = ArrangePopulation(population)    
    print(f"\nFinal sorted population (Top {SelectionSize}): {population}")
    print(f"\nFinal sorted population scores (Top {SelectionSize}): {[gene.GeneralScore+gene.ReasoningScore for gene in population]}")
    return population[:SelectionSize]